In [1]:
from theano.sandbox import cuda

Using gpu device 0: Tesla K80 (CNMeM is disabled)


In [2]:
%matplotlib inline
import utils_modified; reload(utils_modified)
from utils_modified import *
from __future__ import division, print_function

Using Theano backend.


In [3]:
import numpy as np
import random
import sys

In [4]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, Embedding, Activation, LSTM, merge, Flatten, Dropout, Lambda
from keras.models import Model, Sequential
from keras.engine.topology import Merge
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD, RMSprop, Adam
from keras.layers.convolutional import *
from keras.utils.data_utils import get_file

In [64]:
# https://keras.io/getting-started/sequential-model-guide/

In [158]:
# https://github.com/fchollet/keras/blob/master/examples/lstm_text_generation.py
path = get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open(path).read()#.lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

corpus length: 600901
total chars: 85
nb sequences: 200287


In [159]:
lag = 6

In [247]:
idx = [char_indices[c] for c in text]

Xs = []
for l in range(lag):
    cdat = [idx[i+l] for i in xrange(0, len(idx)-1-lag, lag)]
    X = np.stack(cdat[:-lag])
    Xs.append(X)

cdat = [idx[i+(l+1)] for i in xrange(0, len(idx)-1-lag, lag)]
Y = np.stack(cdat[:-lag])

In [280]:
hdim = 250

inputs = [Input(shape=(1,), dtype='int64') for i in range(lag)]

E = Embedding(output_dim=hdim, input_dim=len(chars), input_length=1)

Di2h = Dense(hdim, activation='relu')
Dh2h = Dense(hdim, activation='relu', init='identity')

#hidden = ... CONSTANT ZERO TENSOR IN KERAS ?

e = Di2h(Flatten()(E(inputs[0])))
hidden = e
for i in range(1,lag):
    e = Di2h(Flatten()(E(inputs[i])))
    hidden = merge([e, Dh2h(hidden)], mode='sum')
predictions = Dense(len(chars), activation='softmax')(hidden)

model = Model(input=inputs, output=predictions)

model.compile(optimizer=Adam(1e-3), loss='categorical_crossentropy', metrics=['accuracy'])

In [281]:
model.optimizer.lr = 1e-3
model.fit(Xs, to_categorical(Y,len(chars)), batch_size=100, nb_epoch=12)

Epoch 1/12
150220/150220 [==============================] - 8s - loss: 2.3582 - acc: 0.3340     
Epoch 2/12
150220/150220 [==============================] - 8s - loss: 2.1023 - acc: 0.3971     
Epoch 3/12
150220/150220 [==============================] - 8s - loss: 2.0229 - acc: 0.4173     
Epoch 4/12
150220/150220 [==============================] - 8s - loss: 1.9719 - acc: 0.4301     
Epoch 5/12
150220/150220 [==============================] - 8s - loss: 1.9375 - acc: 0.4370     
Epoch 6/12
150220/150220 [==============================] - 8s - loss: 1.9102 - acc: 0.4442     
Epoch 7/12
150220/150220 [==============================] - 8s - loss: 1.8887 - acc: 0.4488     
Epoch 8/12
150220/150220 [==============================] - 8s - loss: 1.8723 - acc: 0.4524     
Epoch 9/12
150220/150220 [==============================] - 8s - loss: 1.8583 - acc: 0.4555     
Epoch 10/12
150220/150220 [==============================] - 8s - loss: 1.8466 - acc: 0.4579     
Epoch 11/12
150220/150220 [==

In [283]:
def show_top_next(inp):
    pad_inp = (' '*lag)+inp
    idxs = [char_indices[c] for c in pad_inp][-lag:]
    ps = model.predict([np.array([i]) for i in idxs])
    probas = ps[0]/np.sum(ps)
    for i in range(5):
        index = np.random.choice(range(len(chars)), size=None, replace=True, p=probas)
        print(inp+chars[index])
    print('')

In [284]:
show_top_next('this i')
show_top_next('hersel')
show_top_next('himsel')
show_top_next('moral')
show_top_next('moralit')
show_top_next('knowledg')

this in
this is
this if
this in
this io

herselv
herselv
herselv
herseli
hersel,

himselv
himselv
himself
himseld
himself

morali
moral 
moral.
moral!
morali

moraliti
moraliti
morality
moralitu
morality

knowledg:
knowledge
knowledge
knowledge
knowledge

